In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
def connect():
    # Connection to DB
    params = {
            'dbname':
            'user': 
            'password': 
            'host': 
            'port': 
        }
    db_url = f"postgresql://{params['user']}:{params['password']}@{params['host']}:{params['port']}/{params['dbname']}"
    return create_engine(db_url)

In [25]:
engine = connect()
query = """
    SELECT * FROM Price p
    """
df = pd.read_sql(query, engine)
df = df.sort_values(by=['date'], ascending=False)

engine.dispose()
df[df['company_id']==3]

,price_id,company_id,date,open_price,close_price,high_price,low_price,dividends,volume,stock_splits
6275,6276,3,2024-04-22 15:30:00,160.54,160.54,160.64,160.18,None,1151530,None
6274,6275,3,2024-04-22 14:30:00,161.13,160.54,161.14,160.49,None,831123,None
6273,6274,3,2024-04-22 13:30:00,160.88,161.12,161.29,160.71,None,653530,None
6272,6273,3,2024-04-22 12:30:00,160.59,160.88,160.91,160.45,None,551295,None
6271,6272,3,2024-04-22 11:30:00,160.47,160.57,160.65,160.18,None,657511,None
...,...,...,...,...,...,...,...,...,...,...
4188,4189,3,2023-02-10 13:30:00,137.73,137.84,138.16,137.60,None,811528,None
4187,4188,3,2023-02-10 12:30:00,137.31,137.74,137.88,137.29,None,608112,None
4186,4187,3,2023-02-10 11:30:00,137.38,137.30,137.56,137.04,None,710427,None
4185,4186,3,2023-02-10 10:30:00,137.15,137.39,137.52,137.01,None,1006438,None


### Realtime Time Series Analysis of Sentiment Scores (by stock)

In [11]:
engine = connect()
query = """
    SELECT
        p.date AS Date,
        AVG(a.sentiment_score) AS Average_Sentiment_Score,
        c.company_name As Company_Name
    FROM
        Analysis a
    JOIN
        Tweet_Headline th ON a.tweet_id = th.tweet_id
    JOIN
        Price p ON th.company_id = p.company_id AND th.date = p.date
    JOIN
        Company c on p.company_id = c.company_id
    GROUP BY
        p.date,
        c.company_name
    ORDER BY
        p.date;
    """
df = pd.read_sql(query, engine)

engine.dispose()
df.head()

,date,average_sentiment_score,company_name


### Realtime Time Series Analysis of Stock Price (by stock) - put beside (1)

In [13]:
engine = connect()
query = """
    SELECT
        p.date AS Date,
        AVG(p.close_price) AS Average_Stock_Price,
        c.company_name AS Company_Name
    FROM
        Price p
    JOIN
        Company c ON p.company_id = c.company_id
    GROUP BY
        p.date,
        c.company_name
    ORDER BY
        p.date;
    """
df = pd.read_sql(query, engine)

engine.dispose()
df.head()

,date,average_stock_price,company_name
0,2024-04-19 15:55:00,174.58,"Amazon.com, Inc."
1,2024-04-19 15:55:00,164.93,Apple Inc.
2,2024-04-19 15:55:00,480.98,"Meta Platforms, Inc."
3,2024-04-19 15:55:00,399.11,Microsoft Corporation
4,2024-04-19 15:55:00,762.15,NVIDIA Corporation


### Top Mentioned Stocks (for the day or/and week)

In [14]:
engine = connect()
query = """
    SELECT
        c.ticker_symbol AS Stock_Symbol,
        c.company_name AS Company_Name,
        COUNT(*) AS Count_of_Mentions
    FROM
        Tweet_Headline th
    JOIN
        Company c ON th.company_id = c.company_id
    WHERE
        th.date >= current_date - interval '7 days' -- Filter by the past week (you can adjust as needed)
    GROUP BY
        c.ticker_symbol,
        c.company_name
    ORDER BY
        Count_of_Mentions DESC;
    """
df = pd.read_sql(query, engine)

engine.dispose()
df.head()

,stock_symbol,company_name,count_of_mentions
0,TSLA,"Tesla, Inc.",6
1,AMZN,"Amazon.com, Inc.",5


### Most Active Trading Days

In [15]:
engine = connect()
query = """
    SELECT
        p.date AS Date,
        SUM(p.volume) AS Trading_Volume,
        COUNT(th.tweet_id) AS Number_of_Tweets_News
    FROM
        Price p
    LEFT JOIN
        Tweet_Headline th ON p.date = th.date
    GROUP BY
        p.date
    ORDER BY
        SUM(p.volume) DESC;
    """
df = pd.read_sql(query, engine)

engine.dispose()
df.head()

,date,trading_volume,number_of_tweets_news
0,2024-04-19 15:55:00,0,0


### Most Active Trading Days (by stock)

In [18]:
engine = connect()
query = """
    SELECT
        p.date AS Date,
        c.ticker_symbol AS Stock,
        c.company_name AS Company_Name,
        SUM(p.volume) AS Trading_Volume,
        COUNT(th.tweet_id) AS Number_of_Tweets_News
    FROM
        Price p
    JOIN
        Company c ON p.company_id = c.company_id
    LEFT JOIN
        Tweet_Headline th ON p.date = th.date AND p.company_id = th.company_id
    GROUP BY
        p.date,
        c.ticker_symbol,
        c.company_name
    ORDER BY
        SUM(p.volume) DESC;
    """
df = pd.read_sql(query, engine)

engine.dispose()
df.head()

,date,stock,company_name,trading_volume,number_of_tweets_news
0,2024-04-12 09:30:00,AAPL,Apple Inc.,5736048,0
1,2024-04-12 09:50:00,AAPL,Apple Inc.,4380156,0
2,2024-04-12 09:45:00,AAPL,Apple Inc.,4254524,0
3,2024-04-12 09:30:00,TSLA,"Tesla, Inc.",4025716,0
4,2024-04-12 09:35:00,AAPL,Apple Inc.,3727539,0


### Sentiment Word Clouds (by stock)

In [19]:
engine = connect()
query = """
    SELECT
        th.Date AS Date,
        th.content AS Tweets_News_Headline,
        a.sentiment_label AS Sentiment_Label,
        a.sentiment_score AS Sentiment_Score
    FROM
        Tweet_Headline th
    JOIN
        Analysis a ON th.tweet_id = a.tweet_id;
    """
df = pd.read_sql(query, engine)

engine.dispose()
df.head()

,date,tweets_news_headline,sentiment_label,sentiment_score
0,2024-09-30 10:22:52,Why can AMZN P5 deliver outstanding performanc...,positive,0.908985
1,2024-04-21 08:00:00,AI To Take Center Stage in This Week's Big Tec...,neutral,0.999303
2,2024-04-21 04:54:00,10 Undervalued Cyclical Stocks To Invest In Ac...,neutral,0.999799
3,2024-04-21 03:11:00,Bull Market Buys: 3 Dow Stocks to Own for the ...,neutral,0.999781
4,2024-04-20 17:04:00,"Near All-Time Lows, There Is Finally Some Posi...",neutral,0.999646


### Sentiment Analysis by Sector

In [25]:
engine = connect()
query = """
    SELECT
        p.date AS Date,
        c.sector AS Sector,
        AVG(a.sentiment_score) AS Average_Sentiment_Score,
        AVG(p.close_price) AS Average_Price
    FROM
        Price p
    JOIN
        Company c ON p.company_id = c.company_id
    LEFT JOIN
        Tweet_Headline th ON p.date = th.date AND p.company_id = th.company_id
    LEFT JOIN
        Analysis a ON th.tweet_id = a.tweet_id
    GROUP BY
        p.date,
        c.sector
    ORDER BY
        p.date;
    """
df = pd.read_sql(query, engine)

engine.dispose()
df.head()

,date,sector,average_sentiment_score,average_price
0,2024-04-12 09:30:00,Communication Services,None,518.330000
1,2024-04-12 09:30:00,Consumer Cyclical,None,180.075000
2,2024-04-12 09:30:00,Technology,None,498.303333
3,2024-04-12 09:35:00,Communication Services,None,515.130000
4,2024-04-12 09:35:00,Consumer Cyclical,None,180.285000
